# Background on processes and Markov Chains

The [transportation problem](<https://en.wikipedia.org/wiki/Transportation_theory_(mathematics)>) is concerned with finding how much it costs to move mass (represented as a probability measure) to another arrangement or distribution (i.e. another probability measure). Probability measures appear naturally from many sources. Instead of considering a single measure one might index these and consider the notion of process. The simplest processes are [Markov Chains](https://en.wikipedia.org/wiki/Markov_chain) which indexed by positive integers, i.e. [discrete time Markov Chains (DTMC)](https://en.wikipedia.org/wiki/Discrete-time_Markov_chain). We provide and un-pack the following standard defintion for clarity.

***Definition (Discrete Time Markov Chain).***
*A discrete-time Markov chain is a sequence of random variables $X_0, X_1, X_2, \ldots$ with the Markov property, namely that the probability of moving to the next state depends only on the present state and not on the previous states:*  
  
$$\Pr(X_{n+1}=x\mid X_1=x_1, X_2=x_2, \ldots, X_n=x_n) = \Pr(X_{n+1}=x\mid X_n=x_n),$$ 
  
*if both conditional probability are well defined, that is, if $\Pr(X_1=x_1,\ldots,X_n=x_n)>0$.*
*The possible values of $X_i$ form a countable set $S$ called the state space of the chain.*


Let's illustrate this definition by choosing a state space, $S = \{0,1\}$ and assume time-homogeneity or stationarity (i.e. $\Pr(X_{n+1}=x\mid X_n=y) = \Pr(X_n=x\mid X_{n-1}=y)$). This means practically that we can represent the conditional probabilities in a matrix as follows:

$$
\Pr(X_{n+1} = j \mid X_n = i) = 
\begin{bmatrix}
\Pr(X_{n+1} = 0 \mid X_n = 0) & \Pr(X_{n+1} = 1 \mid X_n = 0) \\
\Pr(X_{n+1} = 0 \mid X_n = 1) & \Pr(X_{n+1} = 1 \mid X_n = 1)
\end{bmatrix}
=
\begin{bmatrix}
1/2 & 1/2 \\
1/2 & 1/2
\end{bmatrix}
$$

Here we have selected some probabilities for illustration purposed.


In this context, key observation can be made about optimal transport related to stationary Markov Chains. Namely, optimal transport on stationary distributions alone does not distinguish chains with different dynamics. The following is given in Section 2 of [Optimal Transport for Stationary Markov Chains via Policy Iteration](https://jmlr.csail.mit.edu/papers/v23/21-0519.html). In addition to the chain specified above, consider the following additional chain:


$$
\Pr(X_{n+1} = j \mid X_n = i) = 
\begin{bmatrix}
\Pr(X_{n+1} = 0 \mid X_n = 0) & \Pr(X_{n+1} = 1 \mid X_n = 0) \\
\Pr(X_{n+1} = 0 \mid X_n = 1) & \Pr(X_{n+1} = 1 \mid X_n = 1)
\end{bmatrix}
=
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}
$$

In both cases, one can readily verify the following relations which demonstrate the stationary distribution of these chains are identical.

$$
\begin{bmatrix}
1/2 & 1/2 \\
1/2 & 1/2
\end{bmatrix}
\begin{bmatrix}
1/2 \\
1/2
\end{bmatrix}
= 
\begin{bmatrix}
1/2 \\
1/2
\end{bmatrix},
\ \
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}
\begin{bmatrix}
1/2 \\
1/2
\end{bmatrix}
= 
\begin{bmatrix}
1/2 \\
1/2
\end{bmatrix}
$$

Thus, we compare these chains by solving the transportation problem between their stationary distributions with any allowable cost, we find a zero cost transportation plan. We note, as is noted by the authors that the first chain is IID and while the second is deterministic (after conditioning). Adding a practical dimension, we have coin flipping deciding a state and fixed rules deciding state in the other. It would be desirable to have a way to separate these situations. This is precisely what the work: [Optimal Transport for Stationary Markov Chains via Policy Iteration](https://jmlr.csail.mit.edu/papers/v23/21-0519.html) does.

## Transition Coupling of two Markov Chains

Following [Optimal Transport for Stationary Markov Chains via Policy Iteration](https://jmlr.csail.mit.edu/papers/v23/21-0519.html) we develop and implement their algorithm to resolve the problem demonstrated above. Their approach is to recognize that we can distinguish these stationary chains consider transition coupling that intuitively include more subspace information. This is done by recognizing one can seek so called *transition couplings* which as their name suggests couple transition matrices.

***Definition (Transition Couplings).*** *Let $P$ and $Q$ be transition matrices on finite state spaces $X$ and $Y$, respectively.
A transition matrix $R \in [0, 1]^{d^2\times d^2}$ is a *transition coupling* of $P$ and $Q$ 
if for every paired-state $(x,y) \in X \times Y$, the distribution $R((x,y),\cdot)$ is a coupling
of the distributions $P(x, \cdot)$ and $Q(y,\cdot)$, formally $R((x,y),\cdot) \in \Pi(P(x,\cdot), Q(y,\cdot))$.
Let $\prod_{\text{TC}}(P,Q)$ denote the set of all transition couplings of $P$ and $Q$.*


Let's build on our simplest example from above, where $X = Y = \{0, 1\}$ and

$$
P = 
\begin{bmatrix}
1/2 & 1/2 \\
1/2 & 1/2
\end{bmatrix}, \ \ 

Q = 
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}.
$$

Let $Z = X \times Y = \{(0,0), (0,1), (1, 0), (1,1)\}$ we want coupling of the product which express as $R: Z \to Z$ which we express using rows. 
One straightforward example of a transition coupling of $P$ and $Q$ arises when they are assumed to be independent—that is, the joint transition probabilities are determined under the assumption of independence between $P$ and $Q$.

$$
R = \Pr(z | (x,y))
= \begin{bmatrix}
1/2\cdot 0 & 1/2 \cdot 1 & 1/2 \cdot 0 & 1/2 \cdot 1 \\
1/2\cdot 1 & 1/2 \cdot 0 & 1/2 \cdot 1 & 1/2 \cdot 0 \\
1/2\cdot 0 & 1/2 \cdot 1 & 1/2 \cdot 0 & 1/2 \cdot 1 \\
1/2\cdot 1 & 1/2 \cdot 0 & 1/2 \cdot 1 & 1/2 \cdot 0 
\end{bmatrix}
= \begin{bmatrix}
0 & 1/2 &  0 & 1/2  \\
1/2 & 0 &  1/2 & 0  \\
0 & 1/2 &  0 & 1/2  \\
1/2 & 0 &  1/2 & 0 
\end{bmatrix}.
$$

We could perform this with matrix operations applying a change of coordinates and then performing an outer product. One could also consider other representation such as the tensor product, but this one allows the row interpretation.

We verify that $R$ also satisfies the stationarity condition since

$$
\begin{bmatrix}
0 & 1/2 &  0 & 1/2  \\
1/2 & 0 &  1/2 & 0  \\
0 & 1/2 &  0 & 1/2  \\
1/2 & 0 &  1/2 & 0 
\end{bmatrix}
\begin{bmatrix}
1/4 \\
1/4 \\
1/4 \\
1/4
\end{bmatrix}
=\begin{bmatrix}
1/4 \\
1/4 \\
1/4 \\
1/4
\end{bmatrix}.
$$

The *Optimal Transition Coupling (OTC) problem* seeks to minimize the expected cost, defined by a function  $c: X \times Y \rightarrow \mathbb{R}_+$, over all transition couplings of two Markov chains with transition matrices $P$ and $Q$. See [Optimal Transport for Stationary Markov Chains via Policy Iteration](https://jmlr.csail.mit.edu/papers/v23/21-0519.html) and [Alignment and comparison of directed networks via transition couplings of random walks](https://academic.oup.com/jrsssb/article-abstract/87/1/186/7754547?redirectedFrom=fulltext) for more details.